In [27]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
!ls
!chmod +x Miniconda3-py37_4.10.3-Linux-x86_64.sh
%env PYTHONPATH=
!./Miniconda3-py37_4.10.3-Linux-x86_64.sh -b -f -p /usr/local
!which conda
!conda --version

--2022-10-25 12:31:06--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.10.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89026327 (85M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.10.3-Linux-x86_64.sh’

Miniconda3-py37_4.1 100%[===================>]  84.90M   158MB/s    in 0.5s    

2022-10-25 12:31:06 (158 MB/s) - ‘Miniconda3-py37_4.10.3-Linux-x86_64.sh’ saved [89026327/89026327]

environment.yml  Miniconda3-py37_4.10.3-Linux-x86_64.sh  sample_data  src


In [30]:
!conda env create -n DLEB -f ./environment.yml
!source activate DLEB

Solving environment: / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



tk-8.6.11            | 3.0 MB    | : 100% 1.0/1 [00:00<00:00,  5.35it/s]
openssl-1.1.1m       | 2.5 MB    | : 100% 1.0/1 [00:00<00:00,  8.28it/s]
setuptools-58.0.4    | 788 KB    | : 100% 1.0/1 [00:00<00:00, 11.13it/s]
certifi-2020.6.20    | 155 KB    | : 100% 1.0/1 [00:00<00:00, 20.28it/s]
pip-21.2.2           | 1.8 MB    | : 100% 1.0/1 [00:00<00:00,  5.95it/s]
wheel-0.37.1         | 33 KB     | : 100% 1.0/1 [00:00<00:00, 22.71it/s]
ncurses-6.3          | 782 KB    | : 100% 1.0/1 [00:00<00:00,  4.56it/s]
zlib-1.2.11          | 108 KB    | : 100% 1.0/1 [00:00<00:00, 21.40it/s]
readline-8.1.2       | 354 KB    | : 100% 1.0/1 [00:00<00:00, 16.97it/s]
ca-certificates-2022 | 118 KB    | : 100% 1.0/1 [00:00<00:00, 22.58it/s]
python-3.6.13        | 32.5 MB   | : 100% 1.0/1 [00:04<0

In [33]:
import sys
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.python.keras.callbacks import TensorBoard

In [34]:
from keras.layers import wrappers
from src.get_args import *
from src.preproc_input import *
from src.custom_layer import *
from src.print_lyrs import *

ImportError: ignored

In [4]:
import os
print(os.getcwd())#显示当前路径
 
#os.chdir('/media/gfq/dataset/museGAN/exp2_all/att_sw_bmusegan_loss_jazz')
#print(os.getcwd())


/content


In [ ]:
args = GetArgs()
config_file = args.config
outdir = args.outdir
learning_rate = args.lr

model_name = args.model_name
model_format = args.model_format

os.system('mkdir -p ' + outdir)
os.system('mkdir -p ' + outdir + '/' + model_name)

In [ ]:
print_lyrs = args.print_lyrs
if print_lyrs:
	os.system('mkdir -p ' + outdir + '/intermediate_layers') 

config = open(config_file, 'r')
config = json.load(config)


gpu_num = len(tf.config.list_physical_devices('GPU'))

In [ ]:
def build_Model1():
	_input_1 = keras.Input((1, 1))
	_dense_1 = layers.Dense(1,
		use_bias = True,
		kernel_initializer = 'RandomNormal',
		bias_initializer = keras.initializers.Constant(0),
		kernel_constraint = None,
		bias_constraint = None,
		kernel_regularizer = None,
		bias_regularizer = None,
		activity_regularizer = None,
		name = 'lyr_dense_1')(_input_1)
	_dense_1 = layers.Activation('sigmoid')(_dense_1)
	_dense_2 = layers.Dense(1,
		use_bias = True,
		kernel_initializer = 'RandomNormal',
		bias_initializer = keras.initializers.Constant(0),
		kernel_constraint = None,
		bias_constraint = None,
		kernel_regularizer = None,
		bias_regularizer = None,
		activity_regularizer = None,
		name = 'lyr_dense_2')(_dense_1)
	_dense_2 = layers.Activation('sigmoid')(_dense_2)
	_dense_3 = layers.Dense(1,
		use_bias = True,
		kernel_initializer = 'RandomNormal',
		bias_initializer = keras.initializers.Constant(0),
		kernel_constraint = None,
		bias_constraint = None,
		kernel_regularizer = None,
		bias_regularizer = None,
		activity_regularizer = None,
		name = 'lyr_dense_3')(_dense_2)
	_dense_3 = layers.Activation('sigmoid')(_dense_3)

	model = keras.Model(inputs = [_input_1], outputs = [_dense_3], name = 'Model1')
	return model

Model1 = build_Model1()
Model1.summary()

In [ ]:
Models = { 'Model1' : Model1 }

input_info = config['inputs']
label_info = config['outputs']
train_ratio, val_ratio, test_ratio = 0.5, 0.25, 0.25
batchSize = 1
_input_1_dataset = input_info['_input_1']['input_filepath']
_input_1_dataset = TXT2INPUT(_input_1_dataset, outdir + '/_input_1', input_info['_input_1']['delimiter'])
_output_1_label = label_info['_output_1']['label_filepath']
_output_1_label = READ_LABEL(_input_1_dataset, 'txt', _output_1_label, label_info['_output_1']['label_type'], label_info['_output_1']['encoding'], outdir + '/_output_1')

train_idx, val_idx, test_idx = SPLIT_DATA(_input_1_dataset.shape[0], train_ratio, val_ratio, test_ratio)

Model1_optimizer = tf.keras.optimizers.RMSprop(learning_rate = learning_rate)

In [ ]:
minimum_delta = 0.001
patience = 10
def Model1_loss(label, prediction):
	loss = tf.keras.losses.BinaryCrossentropy()(label, prediction)
	return loss

def train_step(data1, label1):
	with tf.GradientTape() as Model1_tape:
		_input_1 = data1

		prediction1 = Model1(_input_1)

		model_inputs = { 'Model1' : [ _input_1 ] }

		Model1_lossval = Model1_loss(prediction1, label1)

	Model1_gradients = Model1_tape.gradient(Model1_lossval, Model1.trainable_variables)
	Model1_optimizer.apply_gradients(zip(Model1_gradients, Model1.trainable_variables))

	return Model1_lossval, model_inputs

def test_step(data1, label1):
	_input_1 = data1

	prediction1 = Model1(_input_1)

	Model1_lossval = Model1_loss(prediction1, label1)

	return Model1_lossval

def train(data1, label1, train_idx, val_idx, batchSize, epochs):
	data1_train = tf.data.Dataset.from_tensor_slices(data1[train_idx]).batch(batchSize)
	label1_train = tf.data.Dataset.from_tensor_slices(label1[train_idx]).batch(batchSize)

	data1_val = tf.data.Dataset.from_tensor_slices(data1[val_idx]).batch(batchSize)
	label1_val = tf.data.Dataset.from_tensor_slices(label1[val_idx]).batch(batchSize)

	for epoch in range(epochs):

		Model1_trainloss, Model1_valloss = 0, 0

		for x_batch1, y_batch1 in zip(data1_train, label1_train):
			Model1_trainloss, model_inputs = train_step(x_batch1, y_batch1)
			if print_lyrs:
				batch_num = 1
				PrintLyrs(print_lyrs, Models, model_inputs, epoch+1, batch_num, outdir)
				batch_num += 1

		for x_batch1, y_batch1 in zip(data1_val, label1_val):
			Model1_valloss = test_step(x_batch1, y_batch1)

		if (epoch+1) % 10 == 0:
			print('Epoch' + str(epoch+1))
			print('Model1_trainloss: ' + str(Model1_trainloss))
			print('Model1_valloss: ' + str(Model1_valloss))

epochs = 100

In [ ]:
with tf.device('/gpu:0'):
	train(_input_1_dataset, _output_1_label, train_idx, val_idx, batchSize, epochs)

data1_test = tf.data.Dataset.from_tensor_slices(_input_1_dataset[test_idx]).batch(batchSize)
label1_test = tf.data.Dataset.from_tensor_slices(_output_1_label[test_idx]).batch(batchSize)

for x_batch1, y_batch1 in zip(data1_test, label1_test):
	Model1_testloss = test_step(x_batch1, y_batch1)
	print('Model1_testloss : ' + str(Model1_testloss))

tf.saved_model.save(Model1, outdir + '/' + model_name + '/' + 'Model1')